In [1]:
import numpy as np
from scipy.linalg import lstsq
import  numpy.random as random
from operator import itemgetter
from scipy.optimize import nnls
import math



In [2]:
# Defining some global parameters

horiz_len=10
num_of_rollouts=10
lamb=0.1

In [3]:
# Defining required classes

class ReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    def flush_all(self):
        self.buffer = []
        self.position = 0
        return

    def push(self, state, action, reward, next_state, done,policy):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done,policy)
        self.position = (self.position + 1) % self.capacity

    def push_batch(self, batch):
        if len(self.buffer) < self.capacity:
            append_len = min(self.capacity - len(self.buffer), len(batch))
            self.buffer.extend([None] * append_len)

        if self.position + len(batch) < self.capacity:
            self.buffer[self.position : self.position + len(batch)] = batch
            self.position += len(batch)
        else:
            self.buffer[self.position : len(self.buffer)] = batch[:len(self.buffer) - self.position]
            self.buffer[:len(batch) - len(self.buffer) + self.position] = batch[len(self.buffer) - self.position:]
            self.position = len(batch) - len(self.buffer) + self.position

    def sample(self, batch_size):
        if batch_size > len(self.buffer):
            batch_size = len(self.buffer)
        batch = random.sample(self.buffer, int(batch_size))
        state, action, reward, next_state, done,policy = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done,policy

    def sample_all_batch(self, batch_size):
        idxes = np.random.randint(0, len(self.buffer), batch_size)
        batch = list(itemgetter(*idxes)(self.buffer))
        state, action, reward, next_state, done,policy = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done,policy

    def return_all(self):
        return self.buffer

    def __len__(self):
        return len(self.buffer)


class ENV:
    def __init__(self,A,B,Q,R,target_state=np.zeros(3)):
        
        self.A=A
        self.B=B
        self.Q=Q
        self.R=R
        self.target_state=target_state
        self.current_action=None
        self.current_state=None
        
    def reset(self):
        self.current_state=np.array([0.4,-0.6,0.2])
        return self.current_state

    def step(self,action):
        mean = self.A@np.array([self.current_state]).T + self.B@action
        next_state=np.random.multivariate_normal(mean.T[0],np.eye(len(mean)))
        part_1=np.array([next_state])@self.Q@np.array([next_state]).T
        part_2=action.T@self.R@action
        
        self.current_action=action
        if np.linalg.norm(next_state-self.target_state)<0.1:
            return part_1+part_2,next_state,True
        
        return part_1+part_2,next_state,False

In [4]:
def New_Estimate(D_real):
    
    # Need to figure out the problem.
    
    if len(D_real)==0:
        return -1
    dim_state=D_real.buffer[0][0].shape[0]
    dim_action=D_real.buffer[0][1].shape[0]
    
    A_T=np.zeros(shape=(dim_state+dim_action,len(D_real)))
    b_T=np.zeros(shape=(dim_state,len(D_real)))
    for i in range(len(D_real)):
        b_T[:,i]=D_real.buffer[i][3]
        A_T[:,i]=np.concatenate((D_real.buffer[i][0],D_real.buffer[i][1].T[0]))
    A=A_T.T
    b=b_T.T
    total_hat=lstsq(A,b)[0] # need to split
    A_hat=total_hat[:dim_state]
    B_hat=total_hat[dim_state:]
    
    A=np.zeros(shape=(len(D_real),dim_state+dim_action))
    b=np.zeros(shape=(len(D_real),1))
    for i in range(len(D_real.buffer)):
        A[i,:]=np.concatenate((D_real.buffer[i][0],D_real.buffer[i][1].T[0]))
        b[i,:]=D_real.buffer[i][2]
    A=np.square(A)
    total_hat=nnls(A,b)[0]
    total_hat=total_hat.T[0]
    Q_hat=total_hat[:dim_state]
    R_hat=total_hat[dim_state:]
    Q_hat=np.diag(Q_hat)
    R_hat=np.diag(R_hat)
    model= [A_hat,B_hat,Q_hat,R_hat]  
    return model

def Sample_state(D_real):
    ind=np.random.randint(low=0,high=D_real.position) 
    return D_real.buffer[ind][0]


def update_D_fake(S_t,model,K_t,D_fake):
    
    A_hat,B_hat,Q_hat,R_hat=model[0],model[1],model[2],model[3]
    
    i=0
    holder=[]
    Is_done=False
    target_state=np.zeros(3)
    
    while i<horiz_len and Is_done!=True:
        prev=S_t
        u_T=K_t@np.array([S_t]).T
        S_t=A_hat@np.array([S_t]).T+B_hat@u_T
        
        next_state=np.random.multivariate_normal(S_t.T[0],np.eye(len(S_t)))
        
        part_1=np.array([next_state])@Q_hat@np.array([next_state]).T
        part_2=u_T.T@R_hat@u_T
        S_t=next_state
        
        if np.linalg.norm(S_t-target_state)<0.01:
            Is_done=True
        
        D_fake.push(prev,u_T,part_1[0][0]+part_2[0][0],next_state,Is_done,K_t)
        i+=1
    
    return 

def update_D_real(K_t,env,D_real):
    
    i=0
    Is_done=False
    while i<horiz_len and Is_done!=True:
        u_T=K_t@np.array([env.current_state]).T
        R_t,S_t,Is_done=env.step(u_T)
        D_real.push(env.current_state,u_T,R_t[0][0],S_t,Is_done,K_t)
        env.current_state=S_t
        if Is_done:
            break
        i+=1  
    return 


def mod_New_Estimate(D_real):
    
    if len(D_real)==0:
        return -1
    dim_state=D_real.buffer[0][0].shape[0]
    dim_action=D_real.buffer[0][1].shape[0]
    A_T=np.zeros(shape=(dim_state+dim_action,len(D_real)))
    b_T=np.zeros(shape=(dim_state,len(D_real)))
    for i in range(len(D_real)):
        b_T[:,i]=D_real.buffer[i][3]
        A_T[:,i]=np.concatenate((D_real.buffer[i][0],D_real.buffer[i][1].T[0]))
    A=A_T.T
    b=b_T.T
    
    # Need to add regulariser term.
    
    lamb = 1
    n_variables = A.shape[1]
    
    opt_parameters=np.zeros(shape=(dim_state,dim_state+dim_action))
    
    for i in range(dim_state):
        A2 = np.concatenate([A, math.sqrt(lamb)*np.eye(n_variables)])
        B2 = np.concatenate([b[:,i], np.zeros(n_variables)])
        opt_parameters[i]=lstsq(A2,B2)[0]
    A_hat = opt_parameters[:,:dim_state]
    B_hat = opt_parameters[:,dim_state:]
    
    A=np.zeros(shape=(len(D_real),dim_state+dim_action))
    b=np.zeros(shape=(len(D_real),1))
    for i in range(len(D_real.buffer)):
        A[i,:]=np.concatenate((D_real.buffer[i][0],D_real.buffer[i][1].T[0]))
        b[i,:]=D_real.buffer[i][2]
    A=np.square(A)
    total_hat=nnls(A,b.T[0])[0]
    Q_hat=np.diag(total_hat[:3])
    R_hat=np.diag(total_hat[3:])
    
    
    
    
    return [A_hat,B_hat,Q_hat,R_hat]


# importance sampling based policy gradient

def Normal_distrb(z, μ, Σ):

    z = np.atleast_2d(z)
    μ = np.atleast_2d(μ)
    Σ = np.atleast_2d(Σ)

    N = z.size

    temp1 = np.linalg.det(Σ) ** (-1/2)
    temp2 = np.exp(-.5 * (z - μ).T @ np.linalg.inv(Σ) @ (z - μ))

    return (2 * np.pi) ** (-N/2) * temp1 * temp2



# Grad computation-Basic form

<img src="/Users/vaishnav/Desktop/Project/MBPO-LQR-main/Screenshot 2022-12-20 at 1.00.21 PM.png"/>

# Second term:

<img src="/Users/vaishnav/Desktop/Project/MBPO-LQR-main/grad_term.jpeg"/>

In [5]:
def get_episodes(D_fake):
    i=0
    globl=[]
    locl=[]
    j=0
    for i in range(D_fake.position):
       locl.append(D_fake.buffer[i])
       flag=D_fake.buffer[i][4]
       i+=1
       j+=1
       if flag or j==horiz_len:
           globl.append(locl)
           locl=[]
           j=0
    return globl

# Term 1 
def Get_importance_term(episode,K):
    prod=1.0
    for i in range(len(episode)):
        S_t=episode[i][0]
        mean=K@np.array([S_t]).T
        var=np.eye(3)*lamb
        num=Normal_distrb(episode[i][1],mean,var)
        mean=episode[i][5]@np.array([S_t]).T
        var=np.eye(3)*lamb
        den=Normal_distrb(episode[i][1],mean,var)
        prod=prod*num/den
        i+=1
    return prod

# Term 3
def Get_Reward(episode):
    r=0.0
    for i in range(len(episode)):
        r+=episode[i][2]
        i+=1
    return r

# Term 2
def Get_gradient_for_episode(episode,K):
    i=0
    grad_sum=np.zeros_like(K)
    while i<len(episode):
        a_t=episode[i][1]
        s_t=episode[i][0]
        grad_sum+=Get_gradient_for_tup(a_t,s_t,K)
        i+=1
    return grad_sum

def Get_gradient_for_tup(a_t,s_t,K):
    un_normlised=a_t@np.array([s_t])-K@np.array([s_t]).T@np.array([s_t])
    return un_normlised*(1/lamb)

def get_gradient(list_of_episodes,K):
    i=0
    grad_sum=0
    while i<len(list_of_episodes):
        term_1=Get_importance_term(list_of_episodes[i],K)
        term_2=Get_Reward(list_of_episodes[i])
        term_3=Get_gradient_for_episode(list_of_episodes[i],K)
        grad_sum+=term_1*term_2*term_3
        i+=1
    return grad_sum/len(list_of_episodes)    

In [6]:
####   Init STEP #######


# A ->>3*3
# B ->>3*3
# C->eye(3)
# K ->>3*3

################################################################

N=2
E=2
M=2
G=2
horiz_len=10
num_of_rollouts=10

################################################################


# True parameters of the env
np.random.seed(0)
A=0.1*np.diag(np.random.rand(3))

np.random.seed(4)
B=0.1*np.diag(np.random.rand(3))

np.random.seed(1)
Q=np.diag([1,0.1,0.01])

np.random.seed(3)
R=np.diag([1,0.1,0.01])
env=ENV(A,B,Q,R)
env.reset() # Reset

K_star=np.array([[1.71734423, 0.        , 0.        ],
       [0.        , 2.95253427, 0.        ],
       [0.        , 0.        , 1.79613291]])

A_hat=0.1*np.diag(np.random.rand(3))   # Initial theta
B_hat=0.1*np.diag(np.random.rand(3))   # Initial theta
Q_hat=0.1*np.diag(np.ones(3))   # Initial theta
R_hat=0.1*np.diag(np.ones(3))   # Initial theta

# init policy

K=np.random.rand(3,3)   # Initial phi
K_t=K

D_real = ReplayMemory(10000)   # Real data
D_fake = ReplayMemory(10000)  # Fake data


In [ ]:
def Print_model(A=None,B=None,Q=None,R=None):
    
    print("\n****************************************************************\n")
    print("A matrix is \n")
    print(A)
    
    print("\n****************************************************************\n")
    print("B matrix is \n")
    print(B)
    
    print("\n****************************************************************\n")
    print("Q matrix is \n")
    print(Q)
    
    print("\n****************************************************************\n")
    print("R matrix is \n")
    print(R)
    
    return

In [ ]:
D_real.buffer[:3]

In [ ]:
Print_model(A,B,Q,R)

In [ ]:
# Estimation check -Test


env.reset()
D_real.flush_all()
for i in range(1000):
    update_D_real(K_t,env,D_real)   # Update D_real
    if i%200==0:
        A_hat,B_hat,Q_hat,R_hat=mod_New_Estimate(D_real)
        print("\n")
        print(np.linalg.norm(A_hat-A))
        print(np.linalg.norm(B_hat-B))
        print(np.linalg.norm(Q_hat-Q))
        print(np.linalg.norm(R_hat-R))
        


In [ ]:
def Test_Fun(model,A,B,Q,R):
    print(np.linalg.norm(model[0]-A))
    print("\n")
    print(np.linalg.norm(model[1]-B))
    print("\n")
    print(np.linalg.norm(model[2]-Q))
    print("\n")
    print(np.linalg.norm(model[3]-R))
    print("\n")

In [ ]:
# Issue occured: diverging Entries in Matrix,state vector
# Solution : Find best set of param for the environment

In [7]:
####   Init STEP #######


# A ->>3*3
# B ->>3*3
# C->eye(3)
# K ->>3*3

################################################################

N=20
E=20
M=5
G=20
horiz_len=10
num_of_rollouts=10

################################################################


# True parameters of the env
np.random.seed(0)
A=0.1*np.diag(np.random.rand(3))

np.random.seed(4)
B=0.1*np.diag(np.random.rand(3))

np.random.seed(1)
Q=np.diag(np.random.rand(3))

np.random.seed(3)
R=np.diag([0.5,0.01,0])
env=ENV(A,B,Q,R)
env.reset() # Reset


A_hat=0.1*np.diag(np.random.rand(3))   # Initial theta
B_hat=0.1*np.diag(np.random.rand(3))   # Initial theta
Q_hat=0.1*np.diag(np.ones(3))   # Initial theta
R_hat=0.1*np.diag(np.ones(3))   # Initial theta

# init policy

K=np.random.rand(3,3)   # Initial phi
K_t=K

D_real = ReplayMemory(10000)   # Real data
D_fake = ReplayMemory(10000)  # Fake data


In [ ]:
## MBPO algorithm

env.reset()
D_fake.flush_all()
D_real.flush_all()
K=1*np.random.rand(3,3)   # Initial phi
K_t=K
A_hat=0.1*np.diag(np.random.rand(3))   # Initial theta
B_hat=0.1*np.diag(np.random.rand(3))   # Initial theta
Q_hat=0.1*np.diag(np.ones(3))   # Initial theta
R_hat=0.1*np.diag(np.ones(3))   # Initial theta

for n in range(N):
    print("Value for n\t",n)
    print("\n")
    print(np.linalg.norm(K_t-K_star))
    model=mod_New_Estimate(D_real) # Regression
    if model==-1:
        model=[A_hat,B_hat,Q_hat,R_hat]  # start 
    for e in range(E):
        # Update D_real
        update_D_real(K_t,env,D_real)
        for m in range(M):
            S_t=Sample_state(D_real)    # Random sampling
            update_D_fake(S_t,model,K_t,D_fake) # update_D_fake
            # Working till here
        for g in range(G):
            list_of_episodes=get_episodes(D_fake)   # Change representation
            term=get_gradient(list_of_episodes,K_t)
            if n==0:
                K_t=K_t-math.pow(10,-2)*(term+np.random.multivariate_normal(np.zeros(3),0.01*np.eye(3)))   # unKnown parameter (From trajectories -off policy settings)
            else:  
                K_t=K_t-math.pow(10,-2)*(term+0.0) # unKnown parameter (From trajectories -off policy settings)
            

In [15]:
from tqdm import tqdm

In [19]:
## Test-1

# Objective : Will perform policy gradient with true Data. and see where it is heading towrads

env.reset()
D_fake.flush_all()
D_real.flush_all()
K=1*np.random.rand(3,3)   # Initial phi
K_t=K
A_hat=0.1*np.diag(np.random.rand(3))   # Initial theta
B_hat=0.1*np.diag(np.random.rand(3))   # Initial theta
Q_hat=0.1*np.diag(np.ones(3))   # Initial theta
R_hat=0.1*np.diag(np.ones(3))   # Initial theta

E=80
G=80

for e in tqdm(range(E)):
    # Update D_real
    print(len(D_real.buffer))
    update_D_real(K_t,env,D_real)
    
    for g in range(G):
        list_of_episodes=get_episodes(D_real)   # Change representation
        term=get_gradient(list_of_episodes,K_t)
        if e==0:
            K_t=K_t-math.pow(10,-2)*(term+np.random.multivariate_normal(np.zeros(3),0.01*np.eye(3)))   # unKnown parameter (From trajectories -off policy settings)
        else:  
            K_t=K_t-math.pow(10,-2)*(term+0.0) # unKnown parameter (From trajectories -off policy settings)

  1%|▏         | 1/80 [00:00<00:09,  8.35it/s]

0
10


  2%|▎         | 2/80 [00:00<00:13,  5.63it/s]

20


  4%|▍         | 3/80 [00:00<00:18,  4.10it/s]

...

 98%|█████████▊| 78/80 [05:16<00:15,  7.61s/it]

780


 99%|█████████▉| 79/80 [05:24<00:07,  7.71s/it]

790


100%|██████████| 80/80 [05:32<00:00,  4.16s/it]


In [17]:
# compared to where we have started with
np.linalg.norm(K_t-K)

1.3017499708805589

In [18]:
# compared 
np.linalg.norm(K_star-K_t)

3.5965319567478944